In [ ]:
!pip install pandas
!pip install matplotlib
!pip install openpyxl

In [ ]:
import pandas as pd
from math import *
from matplotlib.pyplot import *
from matplotlib import *

In [ ]:
df = pd.read_excel("INFO_Anonyme.xlsx")
categories_a_drop_car_vides =[col for col in df if df[col].isna().all()]
print('Colonnes supprimées :', categories_a_drop_car_vides)
df = df.drop(columns=categories_a_drop_car_vides)
print(df)

In [ ]:
# Fonction qui permet d'afficher un graphe d'une catégorie avec conditions
# catégorie : à prendre sur excel
# minimum : affiche tout les clés qui ont + que le minimum d'étudiant, et affiche ce qui ont - du minimum dans une catégorie "autres"
# Exemple d'utilisation :
# graph_condition("Langue vivante B scolarité - Libellé 2022/2023", 100)
# show()
def graph(categorie, type_graph='bar', minimum=20, classement=[]):
    if(categorie[:7] == 'Moyenne'): type_graph = 'densite'
    if(type_graph == 'densite'): return graph_densite(categorie)
    # Convertir toutes les valeurs en chaînes pour uniformiser
    str_values = df[categorie].astype(str)
    dic = str_values.value_counts()
    X_labels = dic.index.tolist()
    Y = dic.values    
    
    # Si on veut observer un graphique spécique sur le classement
    if(classement != []):
        # Nécéssite deux mises à jours sur X_labels et Y (X_classement sur X_labels et pareil pour Y)
        X_labels = X_classement(categorie, X_labels, classement)
        Y = Y_classement(categorie, X_labels, classement)
        dic = pd.Series(Y, index=X_labels)

    # Mise à jour de X et Y avec les fonctions autres_X et autres_Y
    X_labels = autres_X(X_labels, Y, minimum)
    Y = autres_Y(X_labels, Y, minimum)
        
    # Supprimer les nan si c'est la barre la plus haute
    if X_labels[0] == 'nan' and Y[0] >= max(Y):
        X_labels = X_labels[1:]
        Y = Y[1:]
    
    # Remplacer 'nan' par 'Non renseigné' dans X_labels restants
    X_labels = ['Non renseigné' if label == 'nan' else label for label in X_labels]

    if (type_graph == 'bar'): return graph_bar(categorie, minimum, Y, X_labels, dic)
    if(type_graph == 'circulaire'): return graph_circulaire(Y, X_labels, dic, categorie)


# Fonction qui renvoie une liste avec +n étudiant et le reste dans 'autres' pour X
def autres_X(X, Y, n):
    tab = []
    autre_inclus = False
    for i in range(len(X)):
        if (Y[i] > n): tab.append(X[i])
        else: autre_inclus = True
    if autre_inclus: tab.append('Autre')
    return tab

    
# Fonction qui renvoie une liste avec +n étudiant et le reste dans 'autres' pour Y
def autres_Y(X, Y, n):
    tab = []
    autre = 0
    for valeur in Y:
        if (valeur > n): tab.append(valeur)
        else: autre += valeur
    if (autre > 0): tab.append(autre)
    return tab


# Fonction qui affiche spécifiquement un graphique en bâtons à partir de graph()
def graph_bar(categorie, minimum, Y, X_labels, dic):
    # Si 'Autre' est l'unique barre ou au moins 2 fois plus grande que la plus grande barre, refaire le graphique avec un minimum 2 fois plus petit
    if ('Autre' in X_labels and minimum <= 20 and len(X_labels) < 25 and (len(X_labels) == 1 or Y[-1] >= 2 * max(Y[:-1]))):
        if (minimum > 1): return graph(categorie, type_graph='bar', minimum=minimum // 2)
        else:
            print("Minimum trop petit, graphique de", categorie, "non généré.")
            return False

    # Afficher la série sans la dernière ligne d'information
    print(dic.to_string(max_rows=15))
        
    # Ajuster la taille du graphique en fonction du nombre de barres (1 à 2 fois la largeur standard)
    if len(X_labels) >= 20:
        largeur = min(6.4 * (1 + (len(X_labels) - 20) / 10), 12.8)
        figure(figsize=(largeur, 4.8))
    
    # Utiliser range(len(X_labels)) pour l'axe des X
    X = range(len(X_labels))
    
    # Ajuster les graduations de l'axe Y pour n'avoir que des nombres entiers
    gca().yaxis.set_major_locator(MaxNLocator(integer=True))

    # Mets des jolies couleurs et Autre en noir
    nbr_couleurs = len(Y) - 1 if 'Autre' in X_labels else len(Y)
    cmap = colormaps['hsv'].resampled(nbr_couleurs)
    colors = [cmap(i) for i in range(nbr_couleurs)]
    if ('Autre' in X_labels): colors.append('black')
    
    bar(X, Y, color=colors)
    ylabel(categorie.replace('Libellé en', 'Libellé en\n'))
    xticks(ticks=X, labels=X_labels, rotation=-45, ha='left')
    return True


In [ ]:
if graph('Type Formation - Libellé  2022/2023'): show()

In [ ]:
def graph_circulaire(Y, X_labels, dic, categorie):
    print(dic.to_string(max_rows=15))
    fig, ax = subplots()
    ax.pie(Y, labels=X_labels, autopct='%1.1f%%')
    ylabel(categorie.replace('Libellé en', 'Libellé en\n'), labelpad=200)
    return True

In [ ]:
graph("Origine de la saisie - Libellé en Langue vivante B pour trimestre 2", type_graph='circulaire')

In [ ]:
# Fonction qui retourne les positions des étudiants classé entre 1 et max d'un groupe (0 pour 7415 et 1 pour 142309)
def classement(max, code):
    C = df.loc[(df['Classement'] != 'NC') & (df['Classement'] != 'ENCF')] # Supprime les NC et les ENCF de la colonne de "Classement"
    classement = C.Classement.sort_values() # Créer un tableau trié de la colonne "Classement" par ordre de Classement (premier à dernier)
    position = classement.index # Créer un tableau avec les positions de tout les étudiants classés

    G = df["Groupe candidat - Code"].value_counts()
    groupe = G.index # Créer un tableau avec les clés 7415 et 142309 de la colonne "Groupe candidat"

    tab = []
    for i in position[:(max*2)]: # *2 car il y a 2 groupes différents, donc 2 "classement" différents
        if df['Groupe candidat - Code'][i] == groupe[code]: # Compare le groupe de chaque étudiant
            tab.append(i) # Ajoute si c'est la position de l'étudiant si il est dans le bon groupe
    return tab

def X_classement(categorie, X, rang):
    tab = []
    for i in rang:
        if (df[categorie][i]) in X and (df[categorie][i] not in tab):
            tab.append(df[categorie][i])
    return tab

def Y_classement(categorie, X, rang):
    tab = []
    for i in range(len(X)) :
        nb = 0
        for j in rang:
            if(df[categorie][j] == X[i]): nb += 1
        tab.append(nb)
    return tab

In [ ]:
graph("Candidat boursier - Libellé", type_graph='circulaire', classement=classement(100,1))

In [ ]:
def graph_densite(categorie):
    notes = pd.to_numeric(df[categorie], errors='coerce').dropna()
    hist(notes, bins=20, density=True, alpha=0.6, color='g')
    moyenne = notes.mean()
    axvline(moyenne, color='blue', linestyle='dashed', linewidth=1)
    text(moyenne, ylim()[1], f'Moyenne: {moyenne:.2f}', ha='right', va='top', color='blue')
    xlim(0, 20)
    title(f"Densité de {categorie}")
    ylabel("Densité")
    xlabel("Notes")
    return True

In [ ]:
graph('Moyenne du Candidat en Enseignement scientifique pour trimestre 1')

In [ ]:
for x in df:
    if(graph(x)): show()